# Chata

WARNING: maskrcnn_benchmark.config load global variables - in case of RuntimeError: Error(s) in loading state_dict for GeneralizedRCNN: size mismatch (...) try restart kernel

In [1]:
model_date = "2018_05_08"

In [2]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import importlib
import requests
from io import BytesIO
from PIL import Image
import numpy as np
from demo.predictor_chata import ChataDemo
from demo.predictor import COCODemo

import maskrcnn_benchmark.config as config

In [3]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

In [4]:
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [5]:
#dataset.get_groundtruth(10)
#dataset.get_img_info(10)
#dataset.map_class_id_to_class_name(0)
#dataset.__getitem__(10)[0]

In [6]:
#importlib.reload(config)
#config.cfg

In [7]:
def load(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [8]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

# Chata

In [9]:
from maskrcnn_benchmark.data.datasets.chata import ChataDataset
import imageio as imio
import os

In [10]:
# "train", "test", "val"
charts_dataset = ChataDataset("./datasets/chata/charts", "val")

In [11]:
def save_sample_results(date, chata_demo, dataset, dataset_type="val", img_range=(0,20)):
    
    os.makedirs(f"chata_results/{date}_samples", exist_ok=True)

    for i in range(*img_range):
        test_img = np.array(dataset.__getitem__(i)[0])
        prediction = chata_demo.run_on_opencv_image(test_img)
        #imshow(prediction)
        imio.imwrite(f'chata_results/{date}_samples/{dataset_type}_{i:2d}.jpg', prediction)

In [12]:
config_file = "./configs/chata/e2e_faster_rcnn_R_50_C4_1x_1_gpu_voc_chata.yaml"
config.cfg.merge_from_file(config_file)
config.cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
config.cfg.merge_from_list(["OUTPUT_DIR", "./output/chata/2019_05_08"])

chata_demo = ChataDemo(
    config.cfg,
    min_image_size=800,
    confidence_threshold=0.5,
)

Loading checkpoint from ./output/chata/2019_05_08/model_0175000.pth
[Checkpointer._load_file] Model loading: ./output/chata/2019_05_08/model_0175000.pth


In [13]:
save_sample_results(model_date, chata_demo, charts_dataset)

Labels: {'pie/donat_chart': 0.616784393787384}
Labels: {'scatter_chart': 0.7536885142326355, 'line_chart': 0.7242999076843262}
Labels: {'line_chart': 0.5204306840896606}
Labels: {'box_chart': 0.9997091889381409}
Labels: {'bar_chart': 0.7801631093025208, 'box_chart': 0.6575672626495361, 'scatter_chart': 0.508940577507019}
Labels: {}
Labels: {'pie/donat_chart': 0.8289043307304382, 'line_chart': 0.5512656569480896}
Labels: {}
Labels: {'box_chart': 0.9970706701278687}
Labels: {'line_chart': 0.8134615421295166}
Labels: {'bar_chart': 0.8362742066383362, 'line_chart': 0.560765266418457}
Labels: {'other_chart': 0.9206874370574951, 'scatter_chart': 0.5733978748321533}
Labels: {'box_chart': 0.9937636256217957}
Labels: {'pie/donat_chart': 0.9404959678649902}
Labels: {'bar_chart': 0.8897619247436523, 'other_object': 0.5799522399902344}
Labels: {'box_chart': 0.977256178855896}
Labels: {'box_chart': 0.9996211528778076}
Labels: {'box_chart': 0.8444489240646362, 'scatter_chart': 0.5784417390823364}
La

# Training loss

In [14]:
import re

In [15]:
def print_loss(date, dataset_type="val"):
    os.makedirs(f"./output/chata/{date}", exist_ok=True)
    os.makedirs(f"./chata_results/{date}", exist_ok=True)
    with open(f"./output/chata/{date}/log.txt") as file:
        logs = file.read()

        for loss_type in ["loss", "loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg"]:
            loss = re.findall(f"{loss_type}: [0-9]*.[0-9]* \([0-9]*.[0-9]*", logs)
            loss = [x[len(loss_type)+2:] for x in loss]
            
            iterations = re.findall(f"iter: [0-9]*", logs)
            iterations = [int(x[6:]) for x in iterations]
            
            plt.plot(iterations, [float(x.split("(")[0]) for x in loss])
            plt.plot(iterations, [float(x.split("(")[1]) for x in loss])
            plt.title(loss_type.upper())
            plt.xlabel('Iteration', fontsize=14)
            plt.ylabel('Loss value', fontsize=14)
            plt.savefig(f"./chata_results/{date}/{dataset_type}_{loss_type}.png")
            plt.clf()

In [16]:
print_loss("2019_05_08")

<Figure size 1440x864 with 0 Axes>

# Test on real dataset

In [17]:
import cv2

In [19]:
catalog_names = os.listdir("chata_results/images")
catalogs = [f"chata_results/images/{name}" for name in catalog_names]

In [20]:
for catalog in catalogs:
    os.makedirs(f'{catalog}/{model_date}', exist_ok=True)
    for img_name in os.listdir(catalog):
        if "png" in img_name:
            test_img = cv2.imread(f"{catalog}/{img_name}")
            test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
            #imshow(test_img)
            prediction = chata_demo.run_on_opencv_image(test_img)
            imio.imwrite(f'{catalog}/{model_date}/{img_name}', prediction)

Labels: {}
Labels: {'table': 0.7076946496963501}
Labels: {}
Labels: {'table': 0.6234707832336426}
Labels: {}
Labels: {'table': 0.9276330471038818}
Labels: {'table': 0.6365758776664734}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {'table': 0.9473521113395691}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {'box_chart': 0.7445366978645325}
Labels: {'pie/donat_chart': 0.5329607129096985}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {}
Labels: {'pie/donat_chart': 0.7666131258010864, 'bar_chart': 0.5391120314598083}
Labels: {'box_chart': 0.9521124958